# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import requests
from bs4 import BeautifulSoup
import os
#from matplotlib.venn import venn2

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

In [132]:
# Uploading our excel s (criminalitydata and criminalitydata2) and visualizing the first 5 rows
criminality = 'data/criminalitydata.xlsx'
criminality = pd.read_excel(criminality, skiprows=2)
criminality.head(5)

pop = 'data/population.xlsx'
pop = pd.read_excel(pop, skiprows=2)
pop.head(5)


,Unnamed: 0,Unnamed: 1,2008Q1,2008Q2,2008Q3,2008Q4,2009Q1,2009Q2,2009Q3,2009Q4,...,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4
0,Total,Copenhagen,509861,511725,511686,516962,518574,520659,521397,526918,...,633035,637936,638117,638678,638790,643613,644431,646812,647509,652564
1,NaN,Frederiksberg,93444,93848,93921,95005,95029,95429,95565,96720,...,104118,104351,103677,103284,102940,103782,103608,103940,104094,104801
2,NaN,Dragør,13261,13315,13350,13452,13411,13416,13460,13550,...,14515,14497,14569,14575,14588,14616,14640,14585,14669,14631
3,NaN,Tårnby,40016,40002,40158,40234,40214,40219,40329,40429,...,42785,42757,42670,42698,42658,42664,42723,42819,43042,43129
4,NaN,Albertslund,27602,27564,27596,27817,27706,27807,27739,27887,...,27543,27500,27366,27192,27113,27411,27599,27548,27547,27576


In [133]:
# Cleaning our data
# Deleting the column that has no information
data = [criminality, pop]
for i in data:
    i.drop("Unnamed: 0", axis = 1, inplace = True)


Import your data, either through an API or manually, and load it. 

In [134]:
# Renaming the "Unnamed: 1" to municipalities
for i in data:
    i.rename(columns = {"Unnamed: 1" : "municipalities"}, inplace=True)



# Renaming the rest of the columns in both databases so that they don't start with a number
for n,i in enumerate(data):
    for h in range(2007, 2022+1):
        for j in range(1, 4+1):
            if n == 0:  # This is the first dataset, i.e criminality data
                i.rename(columns={ str(str(h)+"Q"+str(j)):f'crim_{h}Q{j}'}, inplace = True)
            else:
                i.rename(columns={ str(str(h)+"Q"+str(j)):f'pop_{h}Q{j}'}, inplace = True)


# Creating the total per year
for h in range(2007, 2022+1):
    criminality[f'crim_{h}'] = criminality[f'crim_{h}Q1'] + criminality[f"crim_{h}Q2"] + criminality[f"crim_{h}Q3"] + criminality[f"crim_{h}Q4"]
    
for h in range(2008, 2022+1):
    pop[f'pop_{h}'] = pop[f'pop_{h}Q1'] + pop[f"pop_{h}Q2"] + pop[f"pop_{h}Q3"] + pop[f"pop_{h}Q4"]

,municipalities,crim_2007Q1,crim_2007Q2,crim_2007Q3,crim_2007Q4,crim_2008Q1,crim_2008Q2,crim_2008Q3,crim_2008Q4,crim_2009Q1,...,crim_2013,crim_2014,crim_2015,crim_2016,crim_2017,crim_2018,crim_2019,crim_2020,crim_2021,crim_2022
0,Copenhagen,21462.0,24280.0,23553.0,23782.0,21623.0,23201.0,25348.0,24929.0,22285.0,...,102478.0,97983.0,93883.0,102763.0,93446.0,89159.0,93849.0,75750.0,70167.0,81216.0
1,Frederiksberg,1907.0,2085.0,1996.0,2161.0,1801.0,1865.0,2060.0,2245.0,1845.0,...,9910.0,9211.0,8666.0,8395.0,8576.0,7230.0,7217.0,7163.0,6761.0,7798.0
2,Dragør,184.0,183.0,127.0,177.0,152.0,175.0,167.0,194.0,124.0,...,620.0,498.0,591.0,589.0,628.0,509.0,660.0,588.0,508.0,496.0
3,Tårnby,1241.0,1388.0,1371.0,1446.0,1300.0,1500.0,1604.0,1543.0,1421.0,...,7178.0,6991.0,7974.0,8447.0,8350.0,7848.0,7374.0,4656.0,4160.0,5759.0
4,Albertslund,609.0,794.0,744.0,756.0,850.0,829.0,765.0,895.0,753.0,...,3182.0,2779.0,2584.0,2767.0,2538.0,2507.0,2607.0,3373.0,2017.0,2483.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Vesthimmerlands,633.0,621.0,673.0,686.0,642.0,660.0,838.0,898.0,742.0,...,2690.0,2404.0,2249.0,2098.0,1891.0,1877.0,1746.0,1674.0,2101.0,1590.0
98,Aalborg,4047.0,4645.0,4375.0,4891.0,4273.0,4864.0,4854.0,4801.0,4306.0,...,19730.0,18193.0,16403.0,15431.0,16463.0,15491.0,14953.0,11486.0,10359.0,14093.0
99,Unknown municipality,2204.0,1334.0,1378.0,1293.0,1171.0,1145.0,1386.0,1137.0,1244.0,...,9353.0,13471.0,19681.0,22194.0,29325.0,28324.0,35965.0,41711.0,40360.0,41431.0
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Explore each data set

In [145]:
#doing the statistics mean of the total years 

criminality.describe()
pop.describe()

,pop_2008Q1,pop_2008Q2,pop_2008Q3,pop_2008Q4,pop_2009Q1,pop_2009Q2,pop_2009Q3,pop_2009Q4,pop_2010Q1,pop_2010Q2,...,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018,pop_2019,pop_2020,pop_2021,pop_2022
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,...,9.900000e+01,9.900000e+01,9.900000e+01,9.900000e+01,9.900000e+01,9.900000e+01,9.900000e+01,9.900000e+01,9.900000e+01,9.900000e+01
mean,55311.020202,55376.424242,55444.666667,55616.111111,55671.222222,55709.969697,55751.929293,55884.151515,55906.444444,55962.030303,...,2.266742e+05,2.278499e+05,2.293538e+05,2.312550e+05,2.327697e+05,2.339700e+05,2.349436e+05,2.354563e+05,2.363737e+05,2.383427e+05
std,60970.757519,61129.762766,61111.373706,61715.863897,61877.645312,62079.456549,62118.680942,62752.828321,62859.302272,63102.530599,...,2.660580e+05,2.703499e+05,2.745897e+05,2.793437e+05,2.839066e+05,2.883372e+05,2.926284e+05,2.957847e+05,2.982499e+05,3.018201e+05
min,96.000000,94.000000,100.000000,100.000000,96.000000,94.000000,97.000000,103.000000,101.000000,100.000000,...,3.810000e+02,3.720000e+02,3.640000e+02,3.580000e+02,3.240000e+02,3.510000e+02,3.470000e+02,3.460000e+02,3.610000e+02,3.780000e+02
25%,29325.000000,29346.500000,29443.500000,29547.000000,29554.500000,29519.000000,29598.000000,29613.000000,29580.000000,29538.000000,...,1.167940e+05,1.166370e+05,1.171060e+05,1.178540e+05,1.183925e+05,1.191920e+05,1.197100e+05,1.203650e+05,1.210290e+05,1.218035e+05
50%,42817.000000,42720.000000,42746.000000,42761.000000,42807.000000,42743.000000,42759.000000,42720.000000,42768.000000,42820.000000,...,1.706550e+05,1.706920e+05,1.708220e+05,1.714260e+05,1.720690e+05,1.721490e+05,1.719930e+05,1.723870e+05,1.725430e+05,1.729500e+05
75%,59614.500000,59694.000000,59790.500000,59811.500000,59788.500000,59709.000000,59666.500000,59570.500000,59488.000000,59445.500000,...,2.345705e+05,2.353365e+05,2.369450e+05,2.396135e+05,2.409595e+05,2.412980e+05,2.412800e+05,2.398550e+05,2.386730e+05,2.395665e+05
max,509861.000000,511725.000000,511686.000000,516962.000000,518574.000000,520659.000000,521397.000000,526918.000000,528208.000000,530902.000000,...,2.253072e+06,2.293733e+06,2.336757e+06,2.381695e+06,2.425726e+06,2.468077e+06,2.509283e+06,2.536760e+06,2.559198e+06,2.591316e+06


In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [136]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [137]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

NameError: name 'venn2' is not defined

<Figure size 1500x700 with 0 Axes>

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.